creating schema and then creating a table "Fact_Sales"

In [ ]:
CREATE SCHEMA [Sales]
GO

IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='Fact_Sales' AND SCHEMA_NAME(schema_id)='Sales')
CREATE TABLE Sales.Fact_Sales (
    SalesOrderNumber VARCHAR(255) NOT NULL,
    Item VARCHAR(255) NOT NULL,
    OrderDate DATE,
    Year INT,
    Month INT,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    Email VARCHAR(255),
    Quantity INT,
    UnitPrice FLOAT,
    Tax FLOAT
);

loading table from lakehouse and storing that as a view in the warehouse


In [ ]:
CREATE VIEW Sales.Orders
AS
SELECT * FROM [<your-lakehouse-name>].[dbo].[salesorders]

defining a stored procedure to load data from the "view" to the fact_sales table based upon the "orderyear"

In [ ]:
CREATE OR ALTER PROCEDURE Sales.LoadDataFromStaging (@OrderYear INT)
AS
BEGIN
   

     -- Load data into the Fact_Sales table
    INSERT INTO Sales.Fact_Sales (SalesOrderNumber, Item, OrderDate, Year, Month, FirstName, LastName, Email, Quantity, UnitPrice, Tax)
    SELECT 
        SalesOrderNumber, 
        Item, 
        OrderDate, 
        YEAR(OrderDate), 
        MONTH(OrderDate), 
        FirstName, 
        LastName, 
        Email, 
        Quantity, 
        UnitPrice, 
        Tax
    FROM [Sales].[Orders]
    WHERE YEAR(OrderDate) = @OrderYear;

    
END;

executing the stored procedure 

In [ ]:
EXEC Sales.LoadDataFromStaging 2019